In [2]:
cipher_text = """OHPXJPKGYXLPNYCBHWRLBJOAYNCHLYGHABXWZYKHGETUQBIJYZLKNDKUBLQWYJRUFEYBBWHABY
OMBARYPLMETWBMBAXYNNYOZZLKJNUJLLUHYGBGNEUHBWNDGNBQNNGZRGWPOIKTFEZSFGNDKMRU
GEYMFHHOCIREXXKLBVYEBYAYURULXUFUYIJXMQHGFMNAXMAXWEJYAMIGKYMMBABUOBUXRYYEIY
QFBGAPNCKMBAJYPBAJYHFLNZKWITLAJNETNEZQXLFKUEFGABULXUFKIEZBJDKLXLMAIOOXUOZL
FIFAJYPUOPSOZAGKXYBYZEICBGNWTIQAYNSUKWUPULVKYMACOXGATNTTMPNUQMBAYOYFCPZYOL
UCXYBWNKSUHXNDKCOVCLNYOTPWOFXUFAUHXPINRXTBXAXIVTFPEZOXYXGMFLCBONTXLAZIYXMA
RYZMYZGMQAYWKM"""
cipher_text = cipher_text.replace('\n', '')
cipher_text

'OHPXJPKGYXLPNYCBHWRLBJOAYNCHLYGHABXWZYKHGETUQBIJYZLKNDKUBLQWYJRUFEYBBWHABYOMBARYPLMETWBMBAXYNNYOZZLKJNUJLLUHYGBGNEUHBWNDGNBQNNGZRGWPOIKTFEZSFGNDKMRUGEYMFHHOCIREXXKLBVYEBYAYURULXUFUYIJXMQHGFMNAXMAXWEJYAMIGKYMMBABUOBUXRYYEIYQFBGAPNCKMBAJYPBAJYHFLNZKWITLAJNETNEZQXLFKUEFGABULXUFKIEZBJDKLXLMAIOOXUOZLFIFAJYPUOPSOZAGKXYBYZEICBGNWTIQAYNSUKWUPULVKYMACOXGATNTTMPNUQMBAYOYFCPZYOLUCXYBWNKSUHXNDKCOVCLNYOTPWOFXUFAUHXPINRXTBXAXIVTFPEZOXYXGMFLCBONTXLAZIYXMARYZMYZGMQAYWKM'

In [9]:
from collections import defaultdict

patterns = defaultdict(list)

for i in range(len(cipher_text)-2):
    patterns[cipher_text[i:i+3]].append(i)
idx_diff = {}
for i in range(len(cipher_text)-2):
    patterns[cipher_text[i:i+4]].append(i)
idx_diff = {}
for pattern, indices in patterns.items():
    if len(indices) > 1:
        diff = [indices[i+1] - indices[i] for i in range(len(indices)-1)]
        idx_diff[pattern] = diff

idx_diff

{'AYN': [304],
 'HAB': [39],
 'ZLK': [48],
 'NDK': [90, 240],
 'MBA': [12, 120, 24, 126],
 'ARY': [366],
 'NYO': [297],
 'BGN': [210],
 'BWN': [258],
 'ULX': [96],
 'LXU': [96],
 'XUF': [96, 126],
 'XMA': [249],
 'KYM': [135],
 'ABU': [59],
 'BAJ': [5],
 'AJY': [5, 61],
 'JYP': [66],
 'XYB': [60],
 'QAY': [126],
 'UHX': [23],
 'WKM': [0],
 'ULXU': [96],
 'LXUF': [96],
 'BAJY': [5],
 'AJYP': [66]}

3~4개의 알파벳이 2번 이상 나오는 경우 간격을 계산한 것이다. 아주 공통적인 패턴은 없지만, 6의 배수 정도로 생각할 수 있다.  
따라서 6의 약수들을 위주로 coincedence를 계산해보자.

In [10]:
def check_coincidence(key_len):
    cipher_by_keylen = [defaultdict(int) for _ in range(key_len)]
    for i, c in enumerate(cipher_text):
        cipher_by_keylen[i % key_len][c] += 1
    coincidences = []
    for cipher_i in cipher_by_keylen:
        n = sum(cipher_i.values())
        coincidence = sum([f * (f-1) for f in cipher_i.values()]) / (n * (n-1))
        coincidences.append(coincidence)
    return coincidences, 
        



In [15]:
import numpy as np

for key_len in [2,3,6]:
    coincidences = np.array(check_coincidence(key_len))
    print(key_len, coincidences.mean(), coincidences.std())
    
print()
for key_len in range(2, 13):
    coincidences = np.array(check_coincidence(key_len))
    print(key_len, coincidences.mean(), coincidences.std())
    

2 0.04824561403508772 0.00028729027809698895
3 0.05806776062347793 0.006277202954438753
6 0.06891698944330522 0.010033155186569583

2 0.04824561403508772 0.00028729027809698895
3 0.05806776062347793 0.006277202954438753
4 0.04862700228832952 0.005547208172481594
5 0.044723682114986466 0.0030800085633091094
6 0.06891698944330522 0.010033155186569583
7 0.04414127539127539 0.005470341369134515
8 0.048040359519488374 0.009461716321773301
9 0.055750300120048016 0.007214649089865352
10 0.04834870443566096 0.00965956874430145
11 0.04682187730968219 0.007928829180345006
12 0.07051889946626788 0.015093602864379838


확실히 key 길이가 6의 배수일 때 coincidence가 높다. 키 길이를 6으로 가정하고 복호화해보자.

In [16]:
key_len = 6
stats_by_keylen = [defaultdict(int) for _ in range(key_len)]
for i, c in enumerate(cipher_text):
    stats_by_keylen[i % key_len][c] += 1

key = []
for i in range(key_len):
    stats_i = sorted(stats_by_keylen[i].items(), key=lambda x:x[1], reverse=True)
    print(stats_i)
    key.append((ord(stats_i[0][0]) - ord('E')) % 26)
    # print(list(map(lambda x: (x[0], round(x[1]/len(cipher_text)*100,1)), sorted(cipher_by_keylen[i].items(), key=lambda x:x[1], reverse=True))))
print(key)

[('K', 9), ('Y', 9), ('Z', 7), ('U', 7), ('R', 5), ('G', 5), ('X', 5), ('O', 4), ('N', 4), ('T', 4), ('J', 4), ('B', 3), ('I', 3), ('S', 3), ('C', 1), ('H', 1), ('Q', 1), ('A', 1), ('E', 1)]
[('Y', 16), ('L', 7), ('U', 6), ('I', 6), ('M', 6), ('H', 5), ('N', 5), ('Z', 4), ('C', 4), ('G', 3), ('O', 3), ('J', 2), ('W', 2), ('F', 2), ('E', 2), ('B', 1), ('S', 1), ('Q', 1), ('X', 1)]
[('B', 12), ('O', 8), ('F', 7), ('X', 6), ('P', 4), ('Y', 4), ('A', 4), ('K', 4), ('Q', 4), ('R', 4), ('L', 3), ('Z', 3), ('T', 3), ('C', 2), ('V', 2), ('N', 1), ('J', 1), ('M', 1), ('I', 1), ('E', 1), ('H', 1)]
[('X', 10), ('L', 8), ('M', 8), ('B', 7), ('U', 6), ('G', 6), ('T', 6), ('W', 4), ('H', 3), ('K', 3), ('A', 3), ('E', 2), ('V', 2), ('Y', 2), ('J', 1), ('N', 1), ('Q', 1), ('I', 1), ('F', 1), ('P', 1)]
[('N', 11), ('F', 8), ('Y', 7), ('U', 6), ('B', 5), ('M', 5), ('L', 4), ('G', 4), ('I', 4), ('J', 3), ('H', 3), ('X', 3), ('A', 3), ('C', 3), ('O', 2), ('W', 2), ('Q', 1), ('Z', 1), ('P', 1)]
[('A', 16),

In [17]:
plain_text = ''
for i, c in enumerate(cipher_text):
    plain_text += chr((ord(c) - ord('A') - key[i % key_len]) % 26 + ord('a'))

print(plain_text)

inseatembecthefiyalreqfestfoccandioatenoxinatiznsforeheaeshaspubwishedyevertselessdincetserequpstforaroposllsmeneionedehatexerafunntionawityinehesubxissioyswouloberecpivedflvorabwysomedubmiteersdenidedtzkeeptsevarilbleblzcklenrthintsedesirnsnisedeclacedthaeitwaswookinrforabwockciaherasdecurelstripwedesbftmuchxoreefqicieneanothprmandltoryrpquirexentwadthattsesubmttterslgreedeomakeeheirctpheragailabweonawzrldwioeroyawtyfrepbasistfitwecetobedelectpdasthpaes


inseatembecthefiyalreqfestfoccandioatenoxinatiznsforeheaeshaspubwishedyevertselessdincetserequpstforaroposllsmeneionedehatexerafunntionawityinehesubxissioyswouloberecpivedflvorabwysomedubmiteersdenidedtzkeeptsevarilbleblzcklenrthintsedesirnsnisedeclacedthaeitwaswookinrforabwockciaherasdecurelstripwedesbftmuchxoreefqicieneanothprmandltoryrpquirexentwadthattsesubmttterslgreedeomakeeheirctpheragailabweonawzrldwioeroyawtyfrepbasistfitwecetobedelectpdasthpaes  

seatembec, fiyal, reqfest 등이 semtember, final, request라고 예상하면 5번째 key가 잘못된 것을 알 수 있다.

In [22]:
# key = [(ord('K') - ord('E')) % 26, (ord('Y') - ord('E')) % 26, (ord('B') - ord('E')) % 26, (ord('L') - ord('E')) % 26, (ord('N') - ord('E')) % 26, (ord('W') - ord('E')) % 26]
key[4] = (ord('Y') - ord('E')) % 26
plain_text = ''
for i, c in enumerate(cipher_text):
    plain_text += chr((ord(c) - ord('A') - key[i % key_len]) % 26 + ord('a'))

print(plain_text)
print(''.join([chr(k + ord('A')) for k in key]))

inseptemberthefinalrequestforcandidatenominationsfortheaeswaspublishedneverthelesssincetherequestforproposalsmentionedthatextrafunctionalityinthesubmissionswouldbereceivedfavorablysomesubmittersdecidedtokeepthevariableblocklengthinthedesignsnistdeclaredthatitwaslookingforablockcipherassecureastripledesbutmuchmoreefficientanothermandatoryrequirementwasthatthesubmittersagreedtomaketheircipheravailableonaworldwideroyaltyfreebasisifitweretobeselectedastheaes
GUXTUW


즉 키는 "GUXTUW"이고 평문은   "inseptemberthefinalrequestforcandidatenominationsfortheaeswaspublishedneverthelesssincetherequestforproposalsmentionedthatextrafunctionalityinthesubmissionswouldbereceivedfavorablysomesubmittersdecidedtokeepthevariableblocklengthinthedesignsnistdeclaredthatitwaslookingforablockcipherassecureastripledesbutmuchmoreefficientanothermandatoryrequirementwasthatthesubmittersagreedtomaketheircipheravailableonaworldwideroyaltyfreebasisifitweretobeselectedastheaes"이다.